In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install torchmetrics

In [ ]:
import os
import time
import torch.multiprocessing
from copy import deepcopy
from tqdm import tqdm
import shutil
from torch.utils.data import DataLoader, Subset

from parameters import Parameters
from train.trainer import Trainer
from deepfake_datasets.datasets import get_dataloader
from train.early_stopping import EarlyStopping
from train.validate import validate_detection, validate_fully_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_ap, compute_mean_f1, compute_mean_acc_detection, compute_mean_ap_detection, compute_batch_iou, compute_batch_localization_f1, compute_batch_ap, compute_accuracy_detection, compute_average_precision_detection


In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
new_dataset_name = 'train_all_4_datasets'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/combined_training_for_ood/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# here we will only train on a third of each dataset

def keep_only_chosen_dataset(folder_path, dataset_name):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and dataset_name not in file_name:
            os.remove(file_path)

chosen_train_dataset = 'lama'
base_path = '/content/datasets/dolos_data/celebahq/fake/train_all_4_datasets'
subfolders = [
    'images/train', 'images/valid', 'images/test',
    'masks/train', 'masks/valid', 'masks/test'
]

# keep only the images of this specific dataset
for subfolder in subfolders:
    full_path = os.path.join(base_path, subfolder)
    if os.path.exists(full_path):
        keep_only_chosen_dataset(full_path, chosen_train_dataset)

In [ ]:
# set parameters
params = Parameters()

# set experiment name
params.experiment_name = 'training_gradually_added_samples_from_4th'

# set backbone
params.arch = 'CLIP:ViT-L/14,RN50'

# create output dirs
params.create_output_dirs()

new_root_path = '/content/datasets/dolos_data/celebahq/'
new_dataset_name = 'train_all_4_datasets'

params.update_dolos_data_paths(new_root_path, new_dataset_name)

# fix the backbone - to train only the decoder
params.fix_backbone = True

# set the feature layer and the decoder type
params.feature_layer = 'layer20'
params.decoder_type = 'conv-12'

# set the batch size and num threads
params.batch_size = 32
params.num_threads = 8

# set the number of epochs
params.num_iter = 30

# set loss type
params.loss_type = 'bce'

for key, value in params.__dict__.items():
    print(f"{key}: {value}")

experiment_name: training_gradually_added_samples_from_4th
task_type: fully_supervised_localization
data_label: train
arch: CLIP:ViT-L/14,RN50
fix_backbone: True
weight_decay: 0.0
batch_size: 32
num_threads: 8
init_type: normal
init_gain: 0.02
train_dataset: train_all_4_datasets
decoder_type: conv-12
feature_layer: layer20
early_stop_epochs: 5
optim: adam
beta1: 0.9
lr: 0.001
show_loss_freq: 50
num_iter: 30
data_root_path: /content/datasets/dolos_data/celebahq
train_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/train
valid_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/valid
test_fake_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/images/test
train_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/train
valid_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/train_all_4_datasets/masks/valid
test_masks_ground_truth_path: /content/da

In [ ]:
# # get the model, along with its trainer
# model_trainer = Trainer(params)

# # get the trained model on the other 3 datasets, which we will finetune on the 4th
# state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_ldm/best_localization_model_vit+rn50_conv12_iou_73.7741_test_ldm.pth', map_location='cpu')
# model_trainer.model.load_state_dict(state_dict['model'], strict=False)
# model_trainer.model = model_trainer.model.cuda()

In [ ]:
# get the dataloaders
full_train_loader = get_dataloader(params)

val_params = deepcopy(params)
val_params.data_label = 'valid'
val_loader = get_dataloader(val_params) # leave the validation set whole

In [ ]:
torch.cuda.empty_cache()

In [ ]:
num_samples_added = 250 # we will see how well the model learns by gradually adding num_samples_added samples from the 4th dataset
accumulated_samples_indices = []

for round in range(1, len(full_train_loader.dataset)//num_samples_added+1):
    print(f'Round {round}, with {round*num_samples_added} samples of the 4th dataset')

    # get the model, along with its trainer - instantiate a new one for each round for independency
    model_trainer = Trainer(params)

    # get the trained model on the other 3 datasets, which we will finetune on the 4th
    state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_lama/best_localization_model_vit+rn50_conv12_iou_58.0887_test_lama.pth', map_location='cpu')
    model_trainer.model.load_state_dict(state_dict['model'], strict=False)
    model_trainer.model = model_trainer.model.cuda()

    subset_indices = list(range(0, num_samples_added*round)) # already shuffled, only selecting the first needed samples
    subset = Subset(full_train_loader.dataset, subset_indices)
    train_loader = DataLoader(subset, batch_size=32, shuffle=True,
                              num_workers=params.num_threads,
                              pin_memory=torch.cuda.is_available(),
                              persistent_workers=params.num_threads > 0)

    params.experiment_name = f'training_gradually_added_samples_from_4th_round_{round}'
    params.create_output_dirs() # new ones for each training round

    # start the training loop
    early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.001)
    best_metric = 0.0 # iou for localization, ap for detection
    print('Length of training set:', len(train_loader.dataset))
    print('Length of validation set:', len(val_loader.dataset))
    start_time = time.time()
    for epoch in tqdm(range(10)): # only 10 epochs for training, should be enough
        print('Epoch:', epoch)

        epoch_loss = 0
        for data in train_loader:
            model_trainer.total_steps += 1

            model_trainer.set_input(data)
            model_trainer.optimize_parameters()

            if model_trainer.total_steps % params.show_loss_freq == 0:
                epoch_loss += model_trainer.loss.item()
                print(f'Train Loss: {model_trainer.loss.item():.4f} at step {model_trainer.total_steps} \t Iter time: {(time.time() - start_time) / model_trainer.total_steps:.2f}')

        epoch_loss /= len(train_loader.dataset)
        print(f'Average Train Loss: {epoch_loss:.4f}')

        # compute training metrics
        if params.task_type == 'fully_supervised_localization':
            compute_mean_iou(model_trainer.ious, verbose=True, extra_text=f'Train epoch {epoch} ')
            model_trainer.ious = []

            compute_mean_f1(model_trainer.f1_best, model_trainer.f1_fixed, verbose=True, extra_text=f'Train epoch {epoch} ')
            model_trainer.f1_best = []
            model_trainer.f1_fixed = []

            compute_mean_ap(model_trainer.ap, verbose=True, extra_text=f'Train epoch {epoch} ')
            model_trainer.ap = []


        # validate the model
        print('Validation:')
        if params.task_type == 'fully_supervised_localization':
            ious, f1_best, f1_fixed, mean_ap, _ = validate_fully_supervised_localization(model_trainer.model, val_loader, params.train_dataset)

            # compute metrics
            mean_iou = compute_mean_iou(ious, verbose=True, extra_text=f'Validation at epoch {epoch} ')

            mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=f'Validation at epoch {epoch} ')

            mean_ap = compute_mean_ap(mean_ap, verbose=True, extra_text=f'Validation at epoch {epoch} ')

            # save the model if the mean iou is improved
            if mean_iou > best_metric:
                best_metric = mean_iou
                model_trainer.save_model(f'best_localization_model_iou_{mean_iou:.4f}.pth')
                print(f'Best model saved at epoch {epoch}!')

            # check for early stopping
            early_stopping(mean_iou)

        # check if early stopping is triggered
        if early_stopping.early_stop:
            print("Early stopping triggered")
            continue_training = model_trainer.adjust_learning_rate()
            if continue_training:
                print("Continuing training with a learning rate reduced by a factor of 10")
                early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.002) # adjust the delta only once, otherwise stop completely
            else:
                print(f"Early stopping training at epoch {epoch}")
                break
        print()
        torch.cuda.empty_cache()


Round 1, with 250 samples of the 4th dataset
Length of training set: 250
Length of validation set: 300


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
Average Train Loss: 0.0000
Train epoch 0 Mean IOU: 83.14
Train epoch 0 Mean F1 best: 0.9758
Train epoch 0 Mean F1 fixed: 0.9486
Train epoch 0 Mean AP: 0.964
Validation:
Length of dataset:  300



100%|██████████| 10/10 [00:06<00:00,  1.61it/s]


Validation at epoch 0 Mean IOU: 54.35
Validation at epoch 0 Mean F1 best: 0.6994
Validation at epoch 0 Mean F1 fixed: 0.5996
Validation at epoch 0 Mean AP: 0.71


...

Validation at epoch 8 Mean IOU: 73.28
Validation at epoch 8 Mean F1 best: 0.906
Validation at epoch 8 Mean F1 fixed: 0.8505
Validation at epoch 8 Mean AP: 0.9043


 90%|█████████ | 9/10 [19:03<02:07, 127.05s/it]

Best model saved at epoch 8!

Epoch: 9
Train Loss: 0.0169 at step 850 	 Iter time: 1.35
Train Loss: 0.0181 at step 900 	 Iter time: 1.35
Average Train Loss: 0.0000
Train epoch 9 Mean IOU: 91.61
Train epoch 9 Mean F1 best: 0.9966
Train epoch 9 Mean F1 fixed: 0.9953
Train epoch 9 Mean AP: 0.9901
Validation:
Length of dataset:  300



100%|██████████| 10/10 [21:09<00:00, 126.96s/it]

Validation at epoch 9 Mean IOU: 71.57
Validation at epoch 9 Mean F1 best: 0.8764
Validation at epoch 9 Mean F1 fixed: 0.8249
Validation at epoch 9 Mean AP: 0.8904
Early Stopping counter: 1 out of 5



In [ ]:
from google.colab import files
for round in range(1, len(full_train_loader.dataset)//num_samples_added+1):
    model_dir = f'/content/experiments/training_gradually_added_samples_from_4th_round_{round}/models/'
    model_files = sorted(
        [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    )

    best_model = model_files[-1]
    best_model_path = os.path.join(model_dir, best_model)

    shutil.copy(best_model_path, f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_lama/finetune_lama/test_{chosen_train_dataset}_round_{round}_{os.path.basename(best_model_path)}')

    # files.download(best_model_path)

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
